# Neural Network 
## with MNIST Dataset

In [3]:
import numpy as np
import keras
%matplotlib inline
import matplotlib.pyplot as plt



Using TensorFlow backend.


In [4]:
# Load Data
(train_x_orig, train_y_orig), (test_x_orig, test_y_orig) = keras.datasets.mnist.load_data()

In [5]:
# Data Values

m_train = train_x_orig.shape[0]
num_px = train_x_orig.shape[1]
m_test = test_x_orig.shape[0]

print ("Number of training examples: " + str(m_train))
print ("Number of testing examples: " + str(m_test))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_x_orig shape: " + str(train_x_orig.shape))
print ("train_y shape: " + str(train_y_orig.shape))
print ("test_x_orig shape: " + str(test_x_orig.shape))
print ("test_y shape: " + str(test_y_orig.shape))

Number of training examples: 60000
Number of testing examples: 10000
Each image is of size: (28, 28, 3)
train_x_orig shape: (60000, 28, 28)
train_y shape: (60000,)
test_x_orig shape: (10000, 28, 28)
test_y shape: (10000,)


In [6]:
# Prepare Training Labels

digits = 10
examples = train_y_orig.shape[0]

train_y_orig = train_y_orig.reshape(1, examples)

train_y = np.eye(digits)[train_y_orig.astype('int32')]
train_y = train_y.T.reshape(digits, examples)
#print(train_y)

In [7]:
# Prepare Test Labels

m_test = test_y_orig.shape[0]
test_y_orig = test_y_orig.reshape(1, m_test)

test_y = np.eye(digits)[test_y_orig.astype('int32')]
test_y = test_y.T.reshape(digits, m_test)
#print(test_y)

## NN

In [8]:
#Model data

# Convert into shape of (784,60000)
train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1).T  
# Convert into shape of (784,10000)
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1).T

train_x = train_x_flatten / 255.
test_x = test_x_flatten / 255.

print ("train_x's shape: " + str(train_x.shape))
print ("test_x's shape: " + str(test_x.shape))

train_y = train_y
test_y = test_y
print ("train_y's shape: " + str(train_y.shape))
print ("test_y's shape: " + str(test_y.shape))


# train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1).T  
# test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1).T

# train_x = train_x_flatten / 255.
# test_x = test_x_flatten / 255.

# print ("train_x's shape: " + str(train_x.shape))
# print ("test_x's shape: " + str(test_x.shape))

# train_y = train_y
# test_y = test_y
# print ("train_y's shape: " + str(train_y.shape))
# print ("test_y's shape: " + str(test_y.shape))




train_x's shape: (784, 60000)
test_x's shape: (784, 10000)
train_y's shape: (10, 60000)
test_y's shape: (10, 10000)


In [9]:
#Define all the activation functions
def sigmoid(value):
    return 1.0 / ( 1.0+np.exp(-value))

def sigmoid_prime(value):
    return sigmoid(value) * (1- sigmoid(value))

def softmax(value):
    return np.exp(value) / (np.sum(np.exp(value), axis = 0))

def sofmax_prime(value):
    return softmax(value) * (1- softmax(value))

In [10]:
# Cost Function

def compute_cost(Y, Y_hat):
    cost_sum = np.sum(np.multiply(Y, np.log(Y_hat)))
    m = Y.shape[1]
    cost = -(1/m) * cost_sum
    return cost

In [11]:
# Initializing Hyper Parameters

n_x = train_x.shape[0]
n_h = 50
learning_rate = 1
digits = 10
m = train_x.shape[1]
epochs = 1000

In [12]:
print('Input Values : {}'.format(n_x))
print('Hidden Values : {}'.format(n_h))
print(m)
print(train_x.shape)

Input Values : 784
Hidden Values : 50
60000
(784, 60000)


In [13]:
# Initializing Weights and Biasis

W1 = np.random.randn(n_h, n_x)
b1 = np.zeros((n_h, 1))
W2 = np.random.randn(digits, n_h)
b2 = np.zeros((digits, 1))
costs = []

In [14]:
W1.shape

(50, 784)

In [15]:
# Running the Neural Network

for i in range(epochs):
    Z1 = np.matmul(W1,train_x) + b1
    A1 = sigmoid(Z1)     # Activation Sigmoid
    Z2 = np.matmul(W2,A1) + b2
    A2 = softmax(Z2)     # Activation Softmax
    cost = compute_cost(train_y, A2)
    
    dZ2 = A2 - train_y
    dW2 = (1./m) * np.matmul(dZ2, A1.T)
    db2 = (1./m) * np.sum(dZ2, axis=1, keepdims=True)
    
    dA1 = np.matmul(W2.T,dZ2)
    dZ1 = dA1 * sigmoid_prime(Z1)
    dW1 = (1./m) * np.matmul(dZ1, train_x.T)
    db1 = (1./m) * np.sum(dZ1, axis = 1, keepdims=True)
    
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    
    if (i % 100 == 0):
        print("Epoch", i, "cost: ", cost)

print("Final cost:", cost)
    

Epoch 0 cost:  8.117687725289919
Epoch 100 cost:  0.7760177271712653
Epoch 200 cost:  0.5905945628491717
Epoch 300 cost:  0.508889736571268
Epoch 400 cost:  0.45956344992313053
Epoch 500 cost:  0.42538144191433197
Epoch 600 cost:  0.3996534498193591
Epoch 700 cost:  0.3792546871678905
Epoch 800 cost:  0.3624575659012541
Epoch 900 cost:  0.3482218720094239
Epoch 1000 cost:  0.33589266141640334
Epoch 1100 cost:  0.3250376229330625


KeyboardInterrupt: 

In [ ]:
# Save the Biasis and weights because it took ages to calculate on Mac xP
np.savetxt('W1.csv', W1, delimiter=',')
np.savetxt('W2.csv', W2, delimiter=',')
np.savetxt('b1.csv', b1, delimiter=',')
np.savetxt('b2.csv', b2, delimiter=',')

In [ ]:
# Test Accuracy 

Z1 = np.matmul(W1, test_x) + b1
A1 = sigmoid(Z1)
Z2 = np.matmul(W2, A1) + b2
A2 = softmax(Z2)
predictions = np.argmax(A2, axis=0)
labels = np.argmax(test_y, axis=0)

correct = 0 
for i in range(labels.size):
    if predictions[i] == labels[i]:
        correct +=1
print('Test Accuracy : {}'.format(100*correct/labels.size))


## CNN